In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [4]:
df_train = pd.read_csv("../data/raw/train.csv")
df_test = pd.read_csv("../data/raw/test.csv")

### Rename columns

In [ ]:
# lower case column names
df_train.columns = df_train.columns.str.lower()

In [7]:
# rename columns to pythonic names
df_train = df_train.rename(columns={
    'passengerid': 'passenger_id',
    'survived': 'survived',
    'pclass': 'passenger_class',
    'sibsp': 'sibling_spouse_count',
    'parch': 'parent_child_count',
    'ticket': 'ticket_number',
    'embarked': 'embarked_port',
})

### Define unnecessary columns

In [ ]:
cols_to_drop =['name', 'cabin', 'ticket_number', 'passenger_id']

In [18]:
# check column names
df_train.head()

,passenger_id,survived,passenger_class,name,sex,age,sibling_spouse_count,parent_child_count,ticket_number,fare,cabin,embarked_port
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Impute embarked values
Only two missing values, filled in with most common value

In [10]:
df_train["embarked_port"] = df_train["embarked_port"].fillna(df_train["embarked_port"].mode()[0])

In [ ]:
# check missing values
df_train.isnull().sum()

passenger_id              0
survived                  0
passenger_class           0
name                      0
sex                       0
age                     177
sibling_spouse_count      0
parent_child_count        0
ticket_number             0
fare                      0
cabin                   687
embarked_port             0
dtype: int64

### Impute age values
Check correlation with age, grouped by columns with high correlation

In [20]:
# fill missing values in age column with grouped median
df_train["age"] = df_train["age"].fillna(df_train.groupby(["passenger_class", "sibling_spouse_count", "parent_child_count"])["age"].transform("median"))
# fallback for remaining missing values fill with global age median
df_train["age"] = df_train["age"].fillna(df_train["age"].median())

In [21]:
# check missing values
df_train['age'].isnull().sum()

0

In [22]:
df_train.to_csv('../data/processed/train_cleaned.csv', index=False)